In [1]:
import json

import openai
from dotenv import load_dotenv

# OpenAI

In [9]:
def list_models(client):
    for m in client.models.list():
        print(m.id)

def gpt_weather_rpt(client: openai.OpenAI, model: str, messages: list, max_tokens: int):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"},
        max_tokens=max_tokens
    )
    weather_report = ""
    if response.choices[0].finish_reason == "stop":
        weather_report = response.choices[0].message.content
    return weather_report

In [2]:
load_dotenv() # OpenAI API Key is stored in an environment variable
openai_client = openai.OpenAI()

In [5]:
list_models(openai_client)

gpt-4o-mini-audio-preview
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
dall-e-2
gpt-4o-2024-08-06
gpt-4o
o1-mini-2024-09-12
o1-preview-2024-09-12
o1-mini
gpt-4o-2024-11-20
o1-preview
whisper-1
dall-e-3
gpt-4-turbo
gpt-4o-audio-preview
gpt-4o-audio-preview-2024-10-01
babbage-002
omni-moderation-latest
omni-moderation-2024-09-26
tts-1-hd-1106
gpt-4o-2024-05-13
tts-1-hd
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4-turbo-2024-04-09
gpt-4-0613
tts-1
gpt-3.5-turbo-16k
tts-1-1106
davinci-002
gpt-4-turbo-preview
gpt-3.5-turbo-1106
gpt-4o-mini-realtime-preview-2024-12-17
gpt-3.5-turbo-instruct
gpt-4
chatgpt-4o-latest
gpt-4o-realtime-preview-2024-10-01
gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-0125
gpt-4o-audio-preview-2024-12-17
gpt-4o-realtime-preview-2024-12-17
gpt-3.5-turbo
text-embedding-3-large
gpt-4o-realtime-preview
text-embedding-3-small
gpt-4-0125-preview
text-embedding-ada-002
gpt-4-1106-preview


In [6]:
INSTRUCTIONS = (
    "You are an AI assistant specialized in interpreting and describing weather data stored in METAR format. "
    "The METAR format is an XML format that standardizes descriptions of observed weather at airports around the world. "
    "You are expert at reading METAR data for a given location and providing an English description of the weather at that location. "
    "The English weather description should be in the style of TV weatherman."
)
PROMPT = (
    "Briefly describe the weather for the location identified in the following METAR weather report. "
    "Format your response as a JSON object with the following key: 'weather_description'. "
)
LOC = "Toronto"
METAR = (
    """<METAR>
      <raw_text>CXTO 041900Z AUTO M03/M11 RMK AO1 SOG 06 1025 SLP250 T10341108</raw_text>
      <station_id>CXTO</station_id>
      <observation_time>2025-02-04T19:00:00Z</observation_time>
      <latitude>43.667</latitude>
      <longitude>-79.4</longitude>
      <temp_c>-3.4</temp_c>
      <dewpoint_c>-10.8</dewpoint_c>
      <sea_level_pressure_mb>1025</sea_level_pressure_mb>
      <quality_control_flags>
        <auto>TRUE</auto>
        <auto_station>TRUE</auto_station>
      </quality_control_flags>
      <flight_category></flight_category>
      <metar_type>METAR</metar_type>
      <elevation_m>115</elevation_m>
    </METAR>"""
)

In [7]:
msgs = [{"role": "system", "content": INSTRUCTIONS}]
content = [{"type": "text", "text": f"{PROMPT} \n\nLOCATION: {LOC} \n\nMETAR: {METAR}"}]
msgs.append({"role": "user", "content": content})

In [10]:
gpt_weather_rpt(
    client=openai_client,
    model="gpt-4o-mini",
    messages=msgs,
    max_tokens=1000
)

'{\n  "weather_description": "Good evening, Toronto! This evening, we have a chilly snapshot of winter with temperatures hovering around minus 3.4 degrees Celsius. A bit of a bite in the air, as the dew point sits at a frosty minus 10.8 degrees. The pressure is steady at 1025 millibars, indicating stable conditions. So, bundle up if you are heading out, and stay warm!"\n}'